In [1]:
from z3 import *

def euclides(a, b):
    r, rr, s, ss, t, tt = a, b, 1, 0, 0, 1
    print(f"Input:  a: {a}, b: {b}, r: {r}, r': {rr}, s: {s}, s': {ss}, t: {t}, t': {tt}")

    while rr != 0:
        q = r // rr
        r, rr, s, ss, t, tt = rr, r - q * rr, ss, s - q * ss, tt, t - q * tt
        print(f"While:  a: {a}, b: {b}, r: {r}, r': {rr}, s: {s}, s': {ss}, t: {t}, t': {tt}, q: {q}")

    print(f"Final:  a: {a}, b: {b}, r: {r}, r': {rr}, s: {s}, s': {ss}, t: {t}, t': {tt}")
    print(f"Result: r: {r}\n")
    return r

euclides(12,18)
euclides(13,27)
euclides(200,120)

def prove(f):
    with Solver() as s:
        s.add(Not(f))
        if s.check() == sat:
            print("Failed to prove.")
        else:
            print("Proved.")

a = Int('a')
b = Int('b')
r = Int('r')
rr = Int('rr')
s = Int('s')
ss = Int('ss')
t = Int('t')
tt = Int('tt')
q = Int('q')

gcd = Function('gcd', IntSort(), IntSort(), IntSort())
gcdAx1 = ForAll([a], Implies(a >= 0, gcd(a, 0) == a))
gcdAx2 = ForAll([b], Implies(b >= 0, gcd(0, b) == b))
gcdAx3 = ForAll([a, b], Implies(And(a > 0, b > 0), gcd(a, b) == gcd(b, a % b)))

gcdF = And(gcdAx1, gcdAx2, gcdAx3)

preCond = And(a > 0, b > 0)

axioms = And(
    r == a,
    rr == b,
    s == 1,
    ss == 0,
    t == 0,
    tt == 1
)

pre = And(preCond, axioms)

inv = And(
    gcd(a, b) == gcd(r, rr),
    r == a * s + b * t,
    rr == a * ss + b * tt
)

preFinal = (Implies(pre, inv))

exitCond = ForAll([r,rr,s,ss,t,tt], And((rr == 0), inv))

posCond = And(
    r == gcd(a, b),
    r == a * s + b * t
)

posFinal = Implies(exitCond, posCond)

spc = And(preFinal, posFinal)

prove(Implies(gcdF, spc))

Input:  a: 12, b: 18, r: 12, r': 18, s: 1, s': 0, t: 0, t': 1
While:  a: 12, b: 18, r: 18, r': 12, s: 0, s': 1, t: 1, t': 0, q: 0
While:  a: 12, b: 18, r: 12, r': 6, s: 1, s': -1, t: 0, t': 1, q: 1
While:  a: 12, b: 18, r: 6, r': 0, s: -1, s': 3, t: 1, t': -2, q: 2
Final:  a: 12, b: 18, r: 6, r': 0, s: -1, s': 3, t: 1, t': -2
Result: r: 6

Input:  a: 13, b: 27, r: 13, r': 27, s: 1, s': 0, t: 0, t': 1
While:  a: 13, b: 27, r: 27, r': 13, s: 0, s': 1, t: 1, t': 0, q: 0
While:  a: 13, b: 27, r: 13, r': 1, s: 1, s': -2, t: 0, t': 1, q: 2
While:  a: 13, b: 27, r: 1, r': 0, s: -2, s': 27, t: 1, t': -13, q: 13
Final:  a: 13, b: 27, r: 1, r': 0, s: -2, s': 27, t: 1, t': -13
Result: r: 1

Input:  a: 200, b: 120, r: 200, r': 120, s: 1, s': 0, t: 0, t': 1
While:  a: 200, b: 120, r: 120, r': 80, s: 0, s': 1, t: 1, t': -1, q: 1
While:  a: 200, b: 120, r: 80, r': 40, s: 1, s': -1, t: -1, t': 2, q: 1
While:  a: 200, b: 120, r: 40, r': 0, s: -1, s': 3, t: 2, t': -5, q: 2
Final:  a: 200, b: 120, r: 40,